# Income/Expense Sankey Visualization

## Miniconda Setup
```
conda env create --file environment.yml
conda activate income-expense-sankey
```

## Imports

In [56]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go

## CSV Import

In [57]:
csv_path = '/mnt/c/Users/andrin.kessler/Nextcloud/Dokumente/Steuern/2023/transactions.csv'

transactions_frame = pd.read_csv(csv_path, delimiter=';')

transactions_frame['betrag'] = transactions_frame['betrag'].str.replace('CHF ', '')
transactions_frame['betrag'] = transactions_frame['betrag'].str.replace('’', '')
transactions_frame['betrag'] = transactions_frame['betrag'].astype(float)

## Data Processing

In [58]:
income_transactions_frame = transactions_frame[transactions_frame['betrag'] > 0]
expense_transactions_frame = transactions_frame[transactions_frame['betrag'] < 0]

source_arr = []
target_arr = []
value_arr = []

income_categories = income_transactions_frame['category'].unique()
for i in range(1, len(income_categories)+1):
    source_arr.append(i)
    target_arr.append(0)
for income_category in income_categories:
    value_arr.append(income_transactions_frame[income_transactions_frame['category'] == income_category]['betrag'].sum())

expense_categories = expense_transactions_frame['category'].unique()
for i in range(len(income_categories)+1, len(income_categories)+len(expense_categories)+1):
    source_arr.append(0)
    target_arr.append(i)
for expense_category in expense_categories:
    value_arr.append(abs(expense_transactions_frame[expense_transactions_frame['category'] == expense_category]['betrag'].sum()))

## Data Visualization

In [59]:
fig = go.Figure(data=[go.Sankey(
    node = dict(
      pad = 15,
      thickness = 20,
      line = dict(color = "black", width = 0.5),
      label = np.concatenate((["total_income"], income_categories, expense_categories)),
      color = "blue"
    ),
    link = dict(
      source = source_arr,
      target = target_arr,
      value = value_arr
  ))])

fig.update_layout(title_text="Income/Expense Sankey Diagram", font_size=10)
fig.show()